# Loading Files

In [ ]:
# Base
import os

# Data processing
import numpy as np
import pandas as pd

pd.set_option('mode.chained_assignment', None) # turn off some annoying warnings

# Number of avialable cores
nslots = int(os.getenv('NSLOTS', 2))
print('cores:' ,nslots, ', dir:', os.getcwd()) # should be '/gpfs0/shai/users/barryb/link-predict'

# Force reload modules each execution
%load_ext autoreload
%autoreload 2

# Load custom modules
from helper.ml_class import LinkPredict
from helper.ml_class import CustomGroupCV, CustomGroupSplit
from helper.base import load_data

In [ ]:
# Load dataframe
meta, df_initial = load_data(
    path_meta = 'data/processed/networks/subsamples_metadata.csv',
    path_subsample = 'data/processed/networks/subsamples_edge_lists.csv',
    paths_features = ['data/processed/features/features_py.csv',
                      'data/processed/features/features_R.csv'],
)

# Setting important variables

In [48]:
results_path = 'results/'

# Modeling and predicting

### Use case


In [3]:
df = df_initial.copy()

# Drop features
to_drop = ['density', 
           'common_neighbor_centrality',
           'shortest_path_length', 
           'shortest_paths_count', 
           'flow_infomap_HL', 
           'flow_infomap_LL', 
           'modular_centrality_infomap_HL', 
           'modular_centrality_infomap_LL',
           'discrepancy.HL', 
           'discrepancy.LL']
df = df.drop(to_drop, axis=1, errors='ignore')

df['Fisher alpha'] = df['Fisher alpha'].astype('float64')

# TODO: Change this in the previuos steps as it is confusing
df.loc[df['class'] == 'TN', 'class'] = 0
df.loc[df['class'] == 'FN', 'class'] = -1
df.loc[df['class'] == 'TP', 'class'] = 1
df['class'] = df['class'].astype('int64')

# Move 'class' to the end
df = df[[c for c in df if c not in ['class']] + ['class']]

In [4]:
cgs = CustomGroupSplit(group_by='name', 
                       stratify_by='community', 
                       fractions_col = 'fraction',
                       train_size=0.7, 
                       diff_range=None,
                       fractions_train = [1],
                       fractions_test = [0.8], 
                       stratify_groups_train = ['Plant-Seed Dispersers', 'Host-Parasite', 'Plant-Pollinator', 'Plant-Herbivore'],
                       stratify_groups_test = ['Plant-Seed Dispersers', 'Host-Parasite', 'Plant-Pollinator', 'Plant-Herbivore'],
                       drop_isolates=False,
                       drop_existing_links = True,
                       keep_fractions_train=True, # Keep all fractions in train, although they are in fractions_train (will needed later)
                       keep_fractions_test=False,
                       random_state=42,
                       )

train_idx, test_idx  = next(cgs.split(df))
train_link_id, test_link_id  = cgs.get_link_ids(0)

X_train, y_train = LinkPredict().subset_data(df, train_link_id, cast_target=False)
X_test, y_test = LinkPredict().subset_data(df, test_link_id, cast_target=True)

In [5]:
cgc_inner = CustomGroupCV(group_by='name', 
                    stratify_by='community', 
                    fractions_col = 'fraction',
                    n_splits = 3,
                    diff_range=None, 
                    fractions_train = [1],
                    fractions_test = [0.8],
                    stratify_groups_train = ['Plant-Seed Dispersers', 'Host-Parasite', 'Plant-Pollinator', 'Plant-Herbivore'],
                    stratify_groups_test = ['Plant-Seed Dispersers', 'Host-Parasite', 'Plant-Pollinator', 'Plant-Herbivore'],
                    drop_isolates=False,
                    drop_existing_links = True,
                    stratify_test_only_groups=False)

In [6]:
cgc_outer = CustomGroupCV(group_by='name', 
                    stratify_by='community', 
                    fractions_col = 'fraction',
                    n_splits = 5,
                    diff_range=None, 
                    fractions_train = [1],
                    fractions_test = [0.8],
                    stratify_groups_train = ['Plant-Seed Dispersers', 'Host-Parasite', 'Plant-Pollinator', 'Plant-Herbivore'],
                    stratify_groups_test = ['Plant-Seed Dispersers', 'Host-Parasite', 'Plant-Pollinator', 'Plant-Herbivore'],
                    drop_isolates=False,
                    drop_existing_links = True,
                    stratify_test_only_groups=False,
                    keep_fractions_train=True, # Keep all fractions in train, although they are in fractions_train (will needed later)
                    keep_fractions_test=False)

### Train-Test split + Cross Validation

In [ ]:
# ml = LinkPredict('RandomForestClassifier')

# ml.fit(X_train,
#        y_train,
#        dataset_link_id=train_link_id, # might not need this
#        cv=cgc_inner,
#        class_weight='balanced',
#        tuner_name='RandomizedSearchCV',
#        columns_to_ignore=['name', 'link_ID', 'subsample_ID', 'fraction', 'repetition', 'higher_level', 'lower_level', 'weight'])

# ml.multi_plot(X_test, y_test, threshold=0.5, 
#     plots=['confusion_matrix', 'grouped_evaluation', 'roc_curve', 'pr_curve', 'probs_distribution', 'roc_curve_split', 'pr_curve_split', 'grouped_evaluation_split', 'feature_importance'])


### Nested CV

In [ ]:
# ml = LinkPredict('RandomForestClassifier')
# X, y = LinkPredict().subset_data(df, cast_target=False)

# ml.fit(X,
#        y,
#        # dataset_link_id=train_link_id, # might not need this
#        # cv=cgc_inner,
#        cv_inner=cgc_inner,
#        cv_outer=cgc_outer, # Nested cross-validation
#        class_weight='balanced',
#        tuner_name='RandomizedSearchCV',
#        columns_to_ignore=['name', 'link_ID', 'subsample_ID', 'fraction', 'repetition', 'higher_level', 'lower_level', 'weight'])

In [ ]:
# # plot feature importance
# ml.plot_feature_importance(n=90)

# # Plot permutation importance
# ml.plot_permutation_importance(X_test, y_test, n_repeats=10)

In [314]:
# # Save results
# results_df = pd.DataFrame({'fold':[]})
# results_link_ids = cgc_outer.get_link_ids()[1]

# for fold in range(cgc_outer.get_n_splits()):
    
#     test_links = results_link_ids[fold]
#     X_test_subset = X[X['link_ID'].isin(test_links)]#.drop(['community'], axis=1)
#     ml.set_fold_model(fold)
    
#     y_proba = ml.predict_proba(X_test_subset)
#     df_len = len(y_proba)
#     results_fold_df = pd.DataFrame({
#         'fold':[fold]*df_len, 
#         'link_ID':test_links,
#         'type_train':[cgc_outer.stratify_groups_train]*df_len,
#         'type_test':[cgc_outer.stratify_groups_test]*df_len,
#         'frac_train':[cgc_outer.fractions_train]*df_len,
#         'frac_test':[cgc_outer.fractions_test]*df_len,
#         'model':[ml.model_name]*df_len,
#         'y_proba':y_proba,
#     })
#     results_df = pd.concat([results_df, results_fold_df], axis=0)

In [315]:
# results_df.to_csv(results_path+'/raw/results_nested_cv.csv', index=False)

### Compare all community types 

In [16]:
from itertools import combinations

# Set netwrok types
network_types = communities = [
    'Host-Parasite', 'Plant-Pollinator', 'Plant-Seed Dispersers', 'Plant-Herbivore',
]

# Create combinations of network types for training and testing
types_combinations = []
for i in [1]: # remove last arg for all combs
    types_combinations += list(combinations(network_types, i))

types_combinations += [('Plant-Seed Dispersers', 'Plant-Pollinator', 'Plant-Herbivore', 'Host-Parasite')] # Ecological
types_combinations+= [('Plant-Seed Dispersers', 'Plant-Herbivore', 'Host-Parasite')] # Ecological-Non-Plant-Pollinator

In [ ]:
trained_models_types=[]
cgc_list = []

for types_train in types_combinations:
    for types_test in types_combinations:
        
        print('Type train:', types_train, '\nType test:', types_test, '\n')
        
        cgc_inner = CustomGroupCV(group_by='name', 
                            stratify_by='community', 
                            fractions_col = 'fraction',
                            n_splits = 3,
                            diff_range=None, 
                            fractions_train = [1],
                            fractions_test = [0.8],
                            stratify_groups_train = types_train,
                            stratify_groups_test = types_test,
                            drop_isolates=False,
                            drop_existing_links = True,
                            stratify_test_only_groups=False)
        
        cgc_outer = CustomGroupCV(group_by='name', 
                            stratify_by='community', 
                            fractions_col = 'fraction',
                            n_splits = 5,
                            diff_range=None, 
                            fractions_train = [1],
                            fractions_test = [0.8],
                            stratify_groups_train = types_train,
                            stratify_groups_test = types_test,
                            drop_isolates=False,
                            drop_existing_links = True,
                            stratify_test_only_groups=False,
                            keep_fractions_train=True, # Keep all fractions in train, although they are in fractions_train (will needed later)
                            keep_fractions_test=False)

        ml = LinkPredict('RandomForestClassifier')

        X, y = LinkPredict().subset_data(df, cast_target=False)

        # Train
        ml.fit(X,
            y,
            cv_inner=cgc_inner,
            cv_outer=cgc_outer,
            class_weight='balanced',
            tuner_name='RandomizedSearchCV',
            columns_to_ignore=['name', 'link_ID', 'subsample_ID', 'fraction', 'repetition', 'higher_level', 'lower_level', 'weight'])

        # Save model
        ml.types_train = types_train
        ml.types_test = types_test
        trained_models_types.append(ml) 
        cgc_list.append(cgc_outer)
        

In [19]:
# Save results
results_df = pd.DataFrame() #{'fold':[]}

for ml, cgc_outer in zip(trained_models_types, cgc_list):

    results_link_ids = cgc_outer.get_link_ids()[1]

    for fold in range(cgc_outer.get_n_splits()):

        test_links = results_link_ids[fold]
        X_test_subset = X[X['link_ID'].isin(test_links)]
        ml.set_fold_model(fold)
        
        y_proba = ml.predict_proba(X_test_subset)
        df_len = len(y_proba)
        results_run_df = pd.DataFrame({
            'fold':[fold]*df_len, 
            'link_ID':test_links,
            'type_train':[list(cgc_outer.stratify_groups_train)]*df_len,
            'type_test':[list(cgc_outer.stratify_groups_test)]*df_len,
            # 'frac_train':[cgc_outer.fractions_train]*df_len,
            # 'frac_test':[cgc_outer.fractions_test]*df_len,
            # 'model':[ml.model_name]*df_len,
            'y_proba':y_proba,
        })
        results_df = pd.concat([results_df, results_run_df], axis=0)

In [21]:
results_df.to_csv(results_path+'/raw/results_domains.csv', index=False)

### Compare ML algorithms


In [ ]:
# Setting important variables
communities = ['Host-Parasite', 'Plant-Pollinator', 'Plant-Seed Dispersers', 'Plant-Herbivore']
classifiers = ['RandomForestClassifier', 'LogisticRegression', 'XGBClassifier']

# Store trained models in a list
trained_models_algos=[]
cgc_list = []

# Iterate over classifiers
for clf_name in classifiers:

   print('Classifier:', clf_name)
    
   cgc_inner = CustomGroupCV(group_by='name', 
                  stratify_by='community', 
                  fractions_col = 'fraction',
                  n_splits = 3,
                  diff_range=None, 
                  fractions_train = [1],
                  fractions_test = [0.8],
                  stratify_groups_train = communities,
                  stratify_groups_test = communities,
                  drop_isolates=False,
                  drop_existing_links = True,
                  stratify_test_only_groups=False)

   cgc_outer = CustomGroupCV(group_by='name', 
                    stratify_by='community', 
                    fractions_col = 'fraction',
                    n_splits = 5,
                    diff_range=None, 
                    fractions_train = [1],
                    fractions_test = [0.8],
                    stratify_groups_train = communities,
                    stratify_groups_test = communities,
                    drop_isolates=False,
                    drop_existing_links = True,
                    stratify_test_only_groups=False,
                    keep_fractions_train=True, # Keep all fractions in train, although they are in fractions_train (will needed later)
                    keep_fractions_test=False)
   
   ml = LinkPredict(clf_name) # Initialize model

   X, y = LinkPredict().subset_data(df, cast_target=False)
   
   ml.fit(X,
      y,
      cv_inner=cgc_inner,
      cv_outer=cgc_outer, # Nested cross-validation
      class_weight='balanced',
      tuner_name='RandomizedSearchCV',
      columns_to_ignore=['name', 'link_ID', 'subsample_ID', 'fraction', 'repetition', 'higher_level', 'lower_level', 'weight'])

   trained_models_algos.append(ml) # Save model
   cgc_list.append(cgc_outer)

In [200]:
# Save results
results_df = pd.DataFrame()

for ml, cgc_outer in zip(trained_models_algos, cgc_list):

    results_link_ids = cgc_outer.get_link_ids()[1]

    for fold in range(cgc_outer.get_n_splits()):

        test_links = results_link_ids[fold]
        # X_test_subset = X[X['link_ID'].isin(test_links)]
        X_test_subset, y_test_subset = ml.subset_data(df, test_links)
        ml.set_fold_model(fold)

        y_proba = ml.predict_proba(X_test_subset)
        df_len = len(y_proba)

        results_run_df = pd.DataFrame({
            'fold':[fold]*df_len,
            'link_ID':test_links,
            'model':[ml.model_name]*df_len,
            'y_proba':y_proba,
        })

        results_df = pd.concat([results_df, results_run_df], axis=0)

In [201]:
# Voting
voting_df = results_df.groupby(['fold', 'link_ID'])['y_proba'].mean().reset_index() # Compute the mean values
voting_df['model'] = 'Voting'
results_df = pd.concat([results_df, voting_df], axis=0) # Merge with the original DataFrame

In [202]:
results_df.to_csv(results_path+'/raw/results_models.csv', index=False)

In [276]:
# Save results
params_df = pd.DataFrame()

for ml, cgc_outer in zip(trained_models_algos, cgc_list):
    for fold in range(cgc_outer.get_n_splits()):
        ml.set_fold_model(fold)

        best_params = ml.trained_model.best_params_
        params_df = pd.concat([params_df, 
                               pd.DataFrame({'model':[ml.model_name], 
                                             'fold':[fold], 
                                             'params':[best_params], 
                                             'params_dist':[ml.params_dist]})], axis=0)

Distbribution and best parameters for each model

In [287]:
# Initialize an empty dictionary to collect data
data = {}

for ml, cgc_outer in zip(trained_models_algos, cgc_list):
    # Initialize a sub-dictionary for each model
    model_data = {}
    
    for fold in range(cgc_outer.get_n_splits()):
        ml.set_fold_model(fold)

        best_params = ml.trained_model.best_params_
        
        # Update model_data with fold information for each parameter
        for param, best_value in best_params.items():
            simple_param = param.replace('classifier__', '')
            
            if simple_param not in model_data:
                model_data[simple_param] = {'model': ml.model_name, 'parameter': simple_param}
            
            # Adding the best value for each fold
            model_data[simple_param][f'best value (fold {fold + 1})'] = best_value

    # Add model data to the main data dictionary, handling params_dist separately
    for param, values in model_data.items():
        # Extract range for this parameter from params_dist (handle both dict and list of dicts)
        params_dist = ml.params_dist
        if isinstance(params_dist, list):  # If params_dist is a list of dictionaries
            param_range = [d.get(param) for d in params_dist if param in d][0]
        else:  # If params_dist is a dictionary
            param_range = params_dist.get(param)
        
        values['range'] = param_range
        data[(ml.model_name, param)] = values

# Convert the collected data into a DataFrame
params_df = pd.DataFrame.from_dict(data, orient='index').reset_index(drop=True)


In [289]:
params_df.to_csv(results_path+'/raw/params_models.csv', index=False)

### Feature importance


In [203]:
feat_importance_all_df = pd.DataFrame()

for fold in range(cgc_outer.get_n_splits()):
    for ml in trained_models_algos:
        ml.set_fold_model(fold)
        feat_importance_df = ml.feature_importance(X_train=X_train).to_frame(ml.model_name).reset_index().rename(columns={'index':'feature'}).melt(id_vars='feature', var_name='model', value_name='importance')
        feat_importance_df['fold'] = fold
        feat_importance_all_df = pd.concat([feat_importance_all_df, feat_importance_df], axis=0)

feat_importance_all_df.reset_index(drop=True).to_csv(results_path+"/raw/feature_importance_nCV.csv", index=False)

### Transductive

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

res = []

# Save results
results_df = pd.DataFrame()

subsamples = df[(df['fraction'] == 0.8) & (df['community'].isin(['Plant-Pollinator', 'Plant-Seed Dispersers', 'Host-Parasite', 'Plant-Herbivore']))]['subsample_ID'].unique()

# Run random forest for each subsample
for subsample in subsamples:
    
    print('Subsample:', subsample)
    
    # Subset data
    df_subsample = df[df['subsample_ID'] == subsample]

    # Feature selection
    

    # Split data
    X, y = df_subsample.iloc[:,:-1], df_subsample.iloc[:,-1]

    ## Set the training set
    X_train = X.copy()
    y_train = y.copy()

    ## Set the test set
    X_test = X.loc[y != 1]
    y_test = y.loc[X_test.index]

    # Avoid errors during fitting, happens when using cv in some cases
    # X_train, y_train = X_train.reset_index(drop=True), y_train.reset_index(drop=True)

    # Relabel links
    y_train[y_train == -1] = 0
    y_test[y_test == -1] = 1

    # Set a transformer
    preprocessor = LinkPredict().set_transformer(X_train, 
                                                columns_to_ignore=['name', 'link_ID', 'subsample_ID', 'fraction', 'repetition', 'higher_level', 'lower_level', 'weight'],
                                                save=False,
                                                return_preprocessor=True)

    # RandomForestClassifier
    param_dist = {
        'n_estimators': [int(x) for x in np.linspace(start = 5, stop = 100, num = 5)],
        'max_features' : ['sqrt', 'log2'],
        'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)] + [None],
        'min_samples_split': [1 ,2, 3, 4, 5, 10],
        'min_samples_leaf': [1, 2, 3, 4, 5, 10],
        'max_samples': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'max_leaf_nodes': [2, 4, 8, 16, 32, 64, 128],
        'bootstrap': [True, False],
        'criterion' : ["gini", "entropy"],
    }
    param_dist = {f'classifier__{k}': v for k, v in param_dist.items()} # Rename each key in the dictionary to 'classifier__' + key
    rf = RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced')

    # Create pipeline
    pipe = Pipeline(steps=[("preprocessor", preprocessor), 
                            ("classifier", rf)])

    # RandomizedSearchCV
    clf = RandomizedSearchCV(pipe, param_distributions=param_dist, scoring='f1', cv=3, random_state=42, n_jobs=-1)
    clf.fit(X_train.reset_index(drop=True), y_train.reset_index(drop=True))

    res.append(clf)

    y_proba = clf.predict_proba(X_test)[:,1]

    results_single_df = pd.DataFrame({
        'link_ID':X_test['link_ID'],
        'ML_single':y_proba,
    })

    results_df = pd.concat([results_df, results_single_df], axis=0)

In [ ]:
results_df.to_csv(results_path+'/raw/results_ML_by_single_networks.csv', index=False)